In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

### **1.**

Create a program that (1) reads frames from a video, (2) turns the result to grayscale,
and (3) performs Canny edge detection on the image. Display all three stages of processing in one image.

In [18]:
cap = cv.VideoCapture("./assets/video.mp4")
FPS = int(cap.get(cv.CAP_PROP_FPS))

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    gray = cv.cvtColor(cv.cvtColor(frame, cv.COLOR_BGR2GRAY), cv.COLOR_GRAY2BGR)
    edges = cv.cvtColor(cv.Canny(gray, 100, 200), cv.COLOR_GRAY2BGR)
    full = cv.vconcat((frame, gray, edges))

    cv.imshow("Video", cv.resize(full, None, fx=0.25, fy=0.25))
    if cv.waitKey(FPS) == ord("q"):
        break

cap.release()
cv.destroyAllWindows()

### **2.**

Create a program that reads in and displays an image. When the user’s mouse clicks
on the image, read in the corresponding pixel (blue, green, red) values and write
those values as text to the screen at the mouse location.

In [19]:
image = cv.imread("./assets/cat.jpg", cv.IMREAD_COLOR)

win = "Image"
cv.namedWindow(win, cv.WINDOW_AUTOSIZE)


def on_click(event, x, y, *_):
    if event == cv.EVENT_LBUTTONDOWN:
        bgr = image[y, x]
        cv.putText(image, f"{bgr}", (x, y), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2, cv.LINE_AA)


cv.setMouseCallback(win, on_click)

while True:
    cv.imshow(win, image)
    if cv.waitKey(1) == ord("q"):
        break

cv.destroyAllWindows()

### **3.**

Allow the user to select a rectangular region in the image by drawing a rectangle
with the mouse button held down, and highlight the region when the mouse
button is released. Be careful to save an image copy in memory so that your
drawing into the image does not destroy the original values there. The next
mouse click should start the process all over again from the original image.

In a separate window, use the drawing functions to draw a graph in blue, green,
and red for how many pixels of each value were found in the selected box. This
is the color histogram of that color region. Th e x-axis should be eight bins that
represent pixel values falling within the ranges 0–31, 32–63, . . ., 223–255. The
y-axis should be counts of the number of pixels that were found in that bin
range. Do this for each color channel, BGR.

In [20]:
class RectDrawer:
    def __init__(self, image):
        self.image = image
        self.image_copy = image.copy()
        self.drawing = False
        self.ix = -1
        self.iy = -1
        self.hist = np.zeros((300, 300, 3), np.uint8)

    def draw_rect(self, event, x, y, *_):
        if event == cv.EVENT_LBUTTONDOWN:
            self.drawing, self.ix, self.iy = True, x, y
        if event == cv.EVENT_MOUSEMOVE and self.drawing:
            self.image_copy = self.image.copy()
            cv.rectangle(self.image_copy, (self.ix, self.iy), (x, y), (255, 255, 255), thickness=2)
        if event == cv.EVENT_LBUTTONUP:
            self.drawing = False
            if x < self.ix:
                x, self.ix = self.ix, x
            if y < self.iy:
                y, self.iy = self.iy, y
            roi = self.image[self.iy : y, self.ix : x]
            self.hist = self.calc_hist(roi)
            self.image = self.image_copy
            cv.rectangle(self.image_copy, (self.ix, self.iy), (x, y), (255, 255, 255), thickness=2)

    def calc_hist(self, roi):
        fig = plt.figure()
        colors = ("b", "g", "r")
        for i, color in enumerate(colors):
            hist = cv.calcHist([roi], [i], None, [64], [0, 256])
            plt.plot(hist, color=color)
        fig.canvas.draw()
        data = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)  # type: ignore
        cols, rows = fig.canvas.get_width_height()
        data = data.reshape(rows, cols, 4)[..., :3]  # Extract only RGB channels
        data = cv.cvtColor(data, cv.COLOR_RGB2BGR)
        plt.close()
        return data

    def draw(self):
        win1 = "Image"
        win2 = "Histogram"

        cv.namedWindow(win1, cv.WINDOW_AUTOSIZE)
        cv.namedWindow(win2, cv.WINDOW_AUTOSIZE)
        cv.setMouseCallback(win1, self.draw_rect)

        while True:
            cv.imshow(win1, self.image_copy)
            cv.imshow(win2, self.hist)
            if cv.waitKey(1) == ord("q"):
                break

        cv.destroyAllWindows()


image = cv.imread("./assets/cat.jpg", cv.IMREAD_COLOR)
drawer = RectDrawer(image)
drawer.draw()

### **4.**

Make an application that reads and displays a video and is controlled by sliders.
One slider will control the position within the video from start to end in 10 
increments; another binary slider should control pause/unpause. Label both sliders
appropriately.

In [21]:
cap = cv.VideoCapture("./assets/video.mp4")
FPS = int(cap.get(cv.CAP_PROP_FPS))
FRAMES = int(cap.get(cv.CAP_PROP_FRAME_COUNT))


win = "Video"
cv.namedWindow(win, cv.WINDOW_AUTOSIZE)
cv.createTrackbar("Frame", win, 0, 10, lambda x: cap.set(cv.CAP_PROP_POS_FRAMES, int(x * FRAMES / 10)))  # type: ignore
cv.createTrackbar("Grayscale", win, 0, 1, lambda x: None)  # type: ignore

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    if cv.getTrackbarPos("Grayscale", win):
        frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        frame = cv.cvtColor(frame, cv.COLOR_GRAY2BGR)

    cv.imshow(win, cv.resize(frame, None, fx=0.25, fy=0.25))
    if cv.waitKey(int(1000 / FPS)) == ord("q"):
        break

cap.release()
cv.destroyAllWindows()